In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_scheduler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.cuda.amp import autocast
from tqdm import tqdm
import numpy as np
import torch.nn as nn

# Load datasets
df_fake = pd.read_csv(r'C:\Users\viraj\Documents\Virajs Projects\Fake News Detection using MAS\Fake.csv')
df_true = pd.read_csv(r'C:\Users\viraj\Documents\Virajs Projects\Fake News Detection using MAS\True.csv')

# Combine datasets
df_fake['label'] = 0  # Label fake news as 0
df_true['label'] = 1  # Label true news as 1
df = pd.concat([df_fake, df_true], ignore_index=True)

# Remove rows with missing values in 'text'
df = df.dropna(subset=['text'])

# Shuffle the dataset to ensure proper mixing
df = df.sample(frac=1).reset_index(drop=True)

# Check if the dataset is balanced
print("Label distribution:\n", df['label'].value_counts())


c:\Users\viraj\Documents\Virajs Projects\Fake News Detection using MAS\mas_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Label distribution:
 label
0    23481
1    21417
Name: count, dtype: int64


In [2]:
# Initialize tokenizer and model
MODEL_NAME = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenize data
MAX_LEN = 84  # Maximum sequence length

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)


c:\Users\viraj\Documents\Virajs Projects\Fake News Detection using MAS\mas_env\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\viraj\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
# Tokenize and prepare datasets
def prepare_data(df, tokenizer, max_len):
    inputs = tokenizer(list(df['text']),
                       padding='max_length',
                       truncation=True,
                       max_length=max_len,
                       return_tensors='pt')
    labels = torch.tensor(df['label'].tolist())
    return TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)

train_data = prepare_data(train_df, tokenizer, MAX_LEN)
val_data = prepare_data(val_df, tokenizer, MAX_LEN)
test_data = prepare_data(test_df, tokenizer, MAX_LEN)

In [4]:
# Create DataLoaders
BATCH_SIZE = 16
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=1)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=train_df['label'])
class_weights = torch.tensor(class_weights, dtype=torch.float)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
class_weights = class_weights.to(device)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\viraj\Documents\Virajs Projects\Fake News Detection using MAS\mas_env\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Create DataLoaders
BATCH_SIZE = 16
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=1)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=train_df['label'])
class_weights = torch.tensor(class_weights, dtype=torch.float)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
class_weights = class_weights.to(device)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=1e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# Loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Training loop
model.train()
for epoch in range(num_epochs):
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")

    for batch in progress_bar:
        inputs, attention_masks, labels = [t.to(device) for t in batch]

        optimizer.zero_grad()  # Reset gradients

        # Forward pass
        outputs = model(inputs, attention_mask=attention_masks)
        logits = outputs.logits

        # Compute loss
        loss = criterion(logits, labels)

        # Backward pass
        loss.backward()

        # Update model parameters
        optimizer.step()
        lr_scheduler.step()

        progress_bar.set_postfix({"loss": loss.item()})

# Validation loop
model.eval()
val_loss = 0
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in val_dataloader:
        inputs, attention_masks, labels = [t.to(device) for t in batch]

        outputs = model(inputs, attention_mask=attention_masks)
        logits = outputs.logits

        # Compute loss
        loss = criterion(logits, labels)
        val_loss += loss.item()

        # Calculate accuracy
        predictions = torch.argmax(logits, dim=-1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

# Validation metrics
avg_val_loss = val_loss / len(val_dataloader)
val_accuracy = correct_predictions / total_predictions
print(f"Validation Loss: {avg_val_loss}")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Save the fine-tuned model and tokenizer
model.save_pretrained('./fine_tuned_xlm_roberta')
tokenizer.save_pretrained('./fine_tuned_xlm_roberta')
print("Model and tokenizer saved successfully.")


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 3: 100%|██████████| 2021/2021 [10:39<00:00,  3.16it/s, loss=6.67e-5]


Validation Loss: 0.00010715113910717062
Validation Accuracy: 100.00%
Model and tokenizer saved successfully.


In [5]:
# Test loop
all_predictions = []
all_labels = []
model.eval()

with torch.no_grad():
    for batch in test_dataloader:
        inputs, attention_masks, labels = [t.to(device) for t in batch]

        with autocast():
            outputs = model(inputs, attention_mask=attention_masks)
            predictions = torch.argmax(outputs.logits, dim=-1)

        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Test metrics
accuracy = accuracy_score(all_labels, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

C:\Users\viraj\AppData\Local\Temp\ipykernel_19788\762266515.py:10: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
c:\Users\viraj\Documents\Virajs Projects\Fake News Detection using MAS\mas_env\lib\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Test Accuracy: 0.5175
Test Precision: 0.0000
Test Recall: 0.0000
Test F1 Score: 0.0000


c:\Users\viraj\Documents\Virajs Projects\Fake News Detection using MAS\mas_env\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Set the device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the fine-tuned XLM-RoBERTa model and tokenizer
MODEL_PATH = r'C:\Users\viraj\Documents\Virajs Projects\Fake News Detection using MAS\fine_tuned_xlm_roberta'
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

# Move the model to the correct device (GPU or CPU)
model.to(device)

# Function to predict if the news is real or fake
def predict(text):
    # Tokenize and encode the input text
    inputs = tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True)  # Specify max_length and truncation

    # Move the inputs to the same device as the model
    inputs = inputs.to(device)

    # Forward pass to get logits
    with torch.no_grad():
        outputs = model(inputs)
        prediction = torch.argmax(outputs.logits, dim=-1)

    return 'Real' if prediction.item() == 1 else 'Fake'


# Example news article snippet
news_article = ('who was')

# Predict
result = predict(news_article)
print(f'The news article is: {result}')


The news article is: Fake


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np

# Custom Dataset Class for loading the input text data
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Agent 5 Class with XLM-RoBERTa
class Agent5:
    def __init__(self, model_path='/content/drive/MyDrive/fine_tuned_xlm_roberta', max_length=256):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.max_length = max_length

    def train(self, train_texts, train_labels, batch_size=8, epochs=3):
        # Convert text data into a Dataset
        train_dataset = NewsDataset(train_texts, train_labels, self.tokenizer, self.max_length)
        train_loader = DataLoader(train_dataset, batch_size=batch_size)

        # Use the AdamW optimizer and cross entropy loss
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=2e-5)
        loss_fn = torch.nn.CrossEntropyLoss()

        # Move model to GPU if available
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(device)

        # Training loop
        self.model.train()
        for epoch in range(epochs):
            running_loss = 0.0
            for batch in train_loader:
                optimizer.zero_grad()

                # Move data to the same device as model
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                # Forward pass
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()

                # Optimizer step
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch {epoch + 1}/{epochs} - Loss: {running_loss / len(train_loader)}')

    def predict(self, text):
        # Prepare the input text for prediction
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']

        # Move data to the same device as model
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Prediction
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

        return 'Fake News' if predictions.item() == 1 else 'Real News'


# Example of using Agent5

# Initialize Agent5 with fine-tuned XLM-RoBERTa
agent5 = Agent5(model_path='/content/drive/MyDrive/fine_tuned_xlm_roberta')

# Sample train data (you should replace this with your actual dataset)
train_texts = ["Breaking news: The stock market crashes!", "Local hero saves the day in small town."]
train_labels = [1, 0]  # 1 for Fake, 0 for Real

# Train the model (use proper data here for actual use)
agent5.train(train_texts, train_labels, epochs=2)

# Example prediction
text_to_predict = "Stock market surges today! Experts predict strong growth."
prediction = agent5.predict(text_to_predict)
print(f"Prediction: {prediction}")


Epoch 1/2 - Loss: 4.666774272918701
Epoch 2/2 - Loss: 3.4869813919067383
Prediction: Real News


In [11]:
#updated agent 5

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F  # For softmax

# Custom Dataset Class for loading the input text data
class NewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Agent 5 Class with XLM-RoBERTa
class Agent5:
    def __init__(self, model_path='/content/drive/MyDrive/fine_tuned_xlm_roberta', max_length=256):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.max_length = max_length

    def train(self, train_texts, train_labels, batch_size=8, epochs=3):
        # Convert text data into a Dataset
        train_dataset = NewsDataset(train_texts, train_labels, self.tokenizer, self.max_length)
        train_loader = DataLoader(train_dataset, batch_size=batch_size)

        # Use the AdamW optimizer and cross-entropy loss
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=2e-5)
        loss_fn = torch.nn.CrossEntropyLoss()

        # Move model to GPU if available
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(device)

        # Training loop
        self.model.train()
        for epoch in range(epochs):
            running_loss = 0.0
            for batch in train_loader:
                optimizer.zero_grad()

                # Move data to the same device as model
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                # Forward pass
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()

                # Optimizer step
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch {epoch + 1}/{epochs} - Loss: {running_loss / len(train_loader)}')

    def predict_with_percentage(self, text):
        # Tokenize and prepare input text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']

        # Move data to the same device as model
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Prediction
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1)

        # Convert probabilities to percentage
        fake_prob = probs[0][0].item() * 100  # Probability of being fake
        real_prob = probs[0][1].item() * 100  # Probability of being real

        return real_prob, fake_prob


# Instantiate Agent5
agent5 = Agent5(model_path='/content/drive/MyDrive/fine_tuned_xlm_roberta')

# Example news article snippet
news_article = "The Prime Minister of India is Criminal"

# Predict with percentages
real_prob, fake_prob = agent5.predict_with_percentage(news_article)

# Print results
print(f'Real News Probability: {real_prob:.2f}%')
print(f'Fake News Probability: {fake_prob:.2f}%')


OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/fine_tuned_xlm_roberta'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
import tweepy

class Agent7:
    def _init_(self, bearer_token):
        """
        Initializes Agent7 with the necessary tools for tweet verification.

        :param bearer_token: Twitter API bearer token for authentication.
        """
        self.tweepy = tweepy  # Store tweepy module as a class attribute
        self.client = tweepy.Client(bearer_token=bearer_token)

    def check_tweet_existence(self, query, max_results=10):
        """
        Checks if any tweets exist for a given query.

        :param query: Search query string (e.g., "AI technology").
        :param max_results: Maximum number of tweets to fetch for verification.
        :return: Boolean indicating if tweets exist, and the fetched tweets.
        """
        search_query = f"{query} -is:retweet lang:en"  # Filter retweets, only English
        try:
            response = self.client.search_recent_tweets(query=search_query, max_results=max_results)
            if response.data:
                tweets = [tweet.text for tweet in response.data]
                return 0, tweets  # Return 0 when tweets are found
            else:
                return 1, []  # Return 1 when no tweets are found
        except self.tweepy.errors.TooManyRequests:
            print("Error: Rate limit exceeded. Please try again later.")
            return 2, []  # Return 2 when rate limit error occurs
        except self.tweepy.errors.Unauthorized as e:
            print("Error: Unauthorized access. Check your bearer token.")
            return 1, []  # Return 1 for unauthorized access (no tweets)
        except Exception as e:
            print(f"Unexpected error while checking tweet existence: {e}")
            return 1, []  # Return 1 for any other errors (no tweets)

    def fetch_and_check(self, query, max_results=10):
        """
        Combines fetching and verifying tweets for a given query.

        :param query: Search query string.
        :param max_results: Maximum number of tweets to fetch for verification.
        :return: Return code and the fetched tweets.
        """
        return self.check_tweet_existence(query, max_results)

'''
# Example Test
if _name_ == "_main_":
    # Replace with your Twitter API bearer token
    bearer_token = "AAAAAAAAAAAAAAAAAAAAADg%2FxAEAAAAASGOF%2Fy04UuLbMC4YH8ohnH6Or68%3DZtofEWGKh1WakS28MbJgrVs7vbx7p9jGuC9vNoKDEmcrIeQdWC"
    agent = Agent7(bearer_token)

    # Test case with a single query
    query = "Destroy india"
    result_code, tweets = agent.fetch_and_check(query)

    if result_code == 0:
        print(f"Tweets found for query '{query}': {tweets}")
    elif result_code == 1:
        print(f"No tweets found for query '{query}'.")
    elif result_code == 2:
        print(f"Rate limit exceeded for query '{query}'. Please try again later.")

'''

In [10]:
import tweepy

class Agent7:
    def __init__(self, bearer_token):
        """
        Initializes Agent7 with the necessary tools for tweet verification.

        :param bearer_token: Twitter API bearer token for authentication.
        """
        self.tweepy = tweepy  # Store tweepy module as a class attribute
        self.client = tweepy.Client(bearer_token=bearer_token)

    def check_tweet_existence(self, query, max_results=10):
        """
        Checks if any tweets exist for a given query.

        :param query: Search query string (e.g., "AI technology").
        :param max_results: Maximum number of tweets to fetch for verification.
        :return: Return code and the fetched tweets.
        """
        search_query = f"{query} -is:retweet lang:en"  # Filter retweets, only English
        try:
            response = self.client.search_recent_tweets(query=search_query, max_results=max_results)
            if response.data:
                tweets = [tweet.text for tweet in response.data]
                return 0, tweets  # Return 0 when tweets are found
            else:
                return 1, []  # Return 1 when no tweets are found
        except self.tweepy.errors.TooManyRequests:
            print("Error: Rate limit exceeded. Please try again later.")
            return 2, []  # Return 2 when rate limit error occurs
        except self.tweepy.errors.Unauthorized as e:
            print("Error: Unauthorized access. Check your bearer token.")
            return 1, []  # Return 1 for unauthorized access (no tweets)
        except Exception as e:
            print(f"Unexpected error while checking tweet existence: {e}")
            return 1, []  # Return 1 for any other errors (no tweets)

    def fetch_and_check(self, query, max_results=10):
        """
        Combines fetching and verifying tweets for a given query.

        :param query: Search query string.
        :param max_results: Maximum number of tweets to fetch for verification.
        :return: None
        """
        # Call the check_tweet_existence function
        return_code, tweets = self.check_tweet_existence(query, max_results)

        # Print the return values
        print(f"Return Code: {return_code}")
        print(f"Fetched Tweets: {tweets}")

        # Allocate dynamic weight or handle scraping issue
        if return_code == 0:
            dynamic_weight = 10
            print(f"Dynamic Weight: {dynamic_weight} (Tweets found)")
        elif return_code == 1:
            dynamic_weight = 0
            print(f"Dynamic Weight: {dynamic_weight} (No tweets found)")
        elif return_code == 2:
            print("Scraping was not possible")


# Replace 'your_bearer_token_here' with your actual Twitter API bearer token.
bearer_token = 'AAAAAAAAAAAAAAAAAAAAADg%2FxAEAAAAASGOF%2Fy04UuLbMC4YH8ohnH6Or68%3DZtofEWGKh1WakS28MbJgrVs7vbx7p9jGuC9vNoKDEmcrIeQdWC'
agent = Agent7(bearer_token)

# Test the functionality
query = "Barbenheimer"
agent.fetch_and_check(query)


Error: Rate limit exceeded. Please try again later.
Return Code: 2
Fetched Tweets: []
Scraping was not possible


In [2]:
import math

class Agent1:
    def __init__(self, case_weight=5, decay_rate=0.1):
        self.case_weight = case_weight
        self.decay_rate = decay_rate
        self.special_symbols = "!@#$%^&*()_+=-[]{}|;:'\",<>?/\\"  # Special symbols

    def check_special_symbols(self, text):
        if not text:  # Handle empty text case
            print("Empty text detected.")
            return 0  # No weight allocated for empty text

        special_count = 0
        total_count = len(text)  # Total number of characters in the text

        for char in text:
            if char in self.special_symbols:
                special_count += 1

        # Calculate the percentage of special symbols
        percentage = (special_count / total_count) * 100

        print(f"Special Symbols: {special_count}, Total Characters: {total_count}, Percentage: {percentage:.2f}%")

        # Apply weight logic
        if percentage <= 20:
            weight = self.case_weight
        else:
            # Exponential decay for percentages above 20%
            reduction_factor = math.exp(-self.decay_rate * (percentage - 20))
            weight = self.case_weight * reduction_factor
            print(f"Reduction Factor: {reduction_factor:.4f}, Adjusted Weight: {weight:.2f}")

        return weight


In [3]:
#!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

class Agent2:
    def __init__(self, case_weight=15):
        self.case_weight = case_weight
        self.analyzer = SentimentIntensityAnalyzer()

    def analyze_sentiment(self, text):
        if not text:  # Handle empty text case
            print("Empty text detected.")
            return 0  # No sentiment detected for empty text

        # Analyze the sentiment of the provided text
        sentiment_score = self.analyzer.polarity_scores(text)
        compound_score = sentiment_score['compound']



        # Calculate percentage based on compound score
        if compound_score == 0:
            # Neutral sentiment
            sentiment_percentage = 50
        elif compound_score > 0:
            # Positive sentiment
            sentiment_percentage = 50 + (compound_score * 50)  # Linear increase from 50% to 100%
        else:
            # Negative sentiment
            sentiment_percentage = 50 + (compound_score * 50)  # Linear decrease from 50% to 0%

        print(f"Sentiment Percentage: {sentiment_percentage:.2f}%")

        # Final weight calculation based on sentiment percentage
        final_weight = (self.case_weight * sentiment_percentage) / 100
        return final_weight



In [4]:
import math

class Agent3:
    def __init__(self, case_weight=5, threshold=10, decay_rate=0.1):
        """
        Initialize Agent3 with parameters for weight, threshold, and decay rate.

        Args:
            case_weight (float): The base weight to assign.
            threshold (float): The percentage threshold for applying decay.
            decay_rate (float): The rate of exponential decay for weights above the threshold.
        """
        self.case_weight = case_weight
        self.threshold = threshold
        self.decay_rate = decay_rate

    def count_hashtags(self, text):
        """
        Count the percentage of words in the text that are hashtags and apply weight logic.

        Args:
            text (str): The input text to analyze.

        Returns:
            float: The adjusted weight based on hashtag percentage and decay logic.
        """
        if not text:  # Handle empty text case
            print("Empty text detected.")
            return 0.0  # No hashtags detected for empty text

        # Split the text into words to calculate the percentage of hashtags
        words = text.split()
        hashtag_count = sum(1 for word in words if word.startswith('#'))

        # Calculate the percentage of hashtags relative to the total number of words
        total_words = len(words)

        if total_words == 0:  # Avoid division by zero
            return 0.0

        hashtag_percentage = (hashtag_count / total_words) * 100

        print(f"Hashtags: {hashtag_count}, Total Words: {total_words}, Percentage: {hashtag_percentage:.2f}%")

        # Apply weight logic based on the threshold
        if hashtag_percentage <= self.threshold:
            weight = self.case_weight
            print(f"Hashtag Percentage below threshold. Weight: {weight}")
        else:
            # Exponential decay for percentages above the threshold
            reduction_factor = math.exp(-self.decay_rate * (hashtag_percentage - self.threshold))
            weight = self.case_weight * reduction_factor
            print(f"Reduction Factor: {reduction_factor:.4f}, Adjusted Weight: {weight:.2f}")

        return weight


In [5]:
import re
import math
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

class Agent4:
    def __init__(self, case_weight, decay_rate=0.1, threshold=10):
        self.case_weight = case_weight  # Base weight for the agent
        self.decay_rate = decay_rate   # How quickly the weight reduces above the threshold
        self.threshold = threshold    # Percentage threshold for decay application
        self.analyzer = SentimentIntensityAnalyzer()
        self.clickbait_keywords = [
            "shocking", "you won’t believe", "what happened next", "this is why",
            "never seen before", "mind-blowing", "amazing", "unbelievable",
            "click here", "must read", "you'll regret not", "read now", "only",
            "warning", "secret", "revealed", "discovered"
        ]

    def check_clickbait(self, text, headline=None, body=None):
        if not text:  # Handle empty text case
            print("Empty text detected.")
            return 0  # No weight allocated for empty text

        # Lowercase for uniformity
        text_lower = text.lower()

        # Check for clickbait keywords
        keyword_count = sum(1 for keyword in self.clickbait_keywords if keyword in text_lower)

        # Check for excessive punctuation
        exclamation_marks = len(re.findall(r'!', text))
        question_marks = len(re.findall(r'\?', text))
        punctuation_score = 1 if exclamation_marks > 2 or question_marks > 1 else 0

        # Check sentiment score
        sentiment_score = self.analyzer.polarity_scores(text)['compound']
        sentiment_clickbait = 1 if sentiment_score <= -0.5 else 0

        # Optional: Compare headline and body
        headline_body_match = 0
        if headline and body:
            body_start = body[:100].lower()
            if headline.lower() in body_start:
                headline_body_match = 1

        # Combine results into a "clickbait score"
        clickbait_score = keyword_count + punctuation_score + sentiment_clickbait + headline_body_match

        # Set theoretical maximum score (4, if all factors are contributing)
        max_possible_score = 4  # Maximum score for the clickbait components considered

        # Calculate clickbait percentage
        if max_possible_score == 0:  # Edge case if max_possible_score is zero
            clickbait_percentage = 0
        else:
            clickbait_percentage = (clickbait_score / max_possible_score) * 100

        print(f"Clickbait Score: {clickbait_score}, Percentage: {clickbait_percentage:.2f}%")

        # Apply weight logic based on clickbait percentage
        if clickbait_percentage <= self.threshold:
            weight = self.case_weight
        else:
            # Apply exponential decay for percentages above the threshold
            reduction_factor = math.exp(-self.decay_rate * (clickbait_percentage - self.threshold))
            weight = self.case_weight * reduction_factor
            print(f"Reduction Factor: {reduction_factor:.4f}, Adjusted Weight: {weight:.2f}")

        return weight


In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

class Agent5:
    def __init__(self, model_path=r'C:\Users\viraj\Documents\Virajs Projects\Fake News Detection using MAS\fine_tuned_xlm_roberta', max_length=256, true_weight=45):
        # Load the pre-trained tokenizer and model from the given path
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.max_length = max_length
        self.true_weight = true_weight  # Assign only the true weight during initialization

    def train(self, train_texts, train_labels, batch_size=8, epochs=3):
        # Convert text data into a Dataset
        train_dataset = NewsDataset(train_texts, train_labels, self.tokenizer, self.max_length)
        train_loader = DataLoader(train_dataset, batch_size=batch_size)

        # Use the AdamW optimizer and cross-entropy loss
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=2e-5)
        loss_fn = torch.nn.CrossEntropyLoss()

        # Move model to GPU if available
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(device)

        # Training loop
        self.model.train()
        for epoch in range(epochs):
            running_loss = 0.0
            for batch in train_loader:
                optimizer.zero_grad()

                # Move data to the same device as model
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                # Forward pass
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()

                # Optimizer step
                optimizer.step()

                running_loss += loss.item()

            print(f'Epoch {epoch + 1}/{epochs} - Loss: {running_loss / len(train_loader)}')

    def predict_true_weight(self, text):
        # Tokenize and prepare input text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']

        # Move data to the same device as model
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Prediction
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        # Apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1)

        # Extract true news probability and calculate weight
        true_prob = probs[0][1].item() * 100  # Probability of being true
        true_weighted = true_prob * self.true_weight / 100  # Weighted true value

        return true_weighted


In [14]:
class Agent7:
    def __init__(self, bearer_token):  # Corrected __init__ (double underscores)
        """
        Initializes Agent7 with the necessary tools for tweet verification.

        :param bearer_token: Twitter API bearer token for authentication.
        """
        self.tweepy = tweepy  # Store tweepy module as a class attribute
        self.client = tweepy.Client(bearer_token=bearer_token)

    def check_tweet_existence(self, query, max_results=10):
        """
        Checks if any tweets exist for a given query.

        :param query: Search query string (e.g., "AI technology").
        :param max_results: Maximum number of tweets to fetch for verification.
        :return: Return code and the fetched tweets.
        """
        search_query = f"{query} -is:retweet lang:en"  # Filter retweets, only English
        try:
            response = self.client.search_recent_tweets(query=search_query, max_results=max_results)
            if response.data:
                tweets = [tweet.text for tweet in response.data]
                return 0, tweets  # Return 0 when tweets are found
            else:
                return 1, []  # Return 1 when no tweets are found
        except self.tweepy.errors.TooManyRequests:
            print("Error: Rate limit exceeded. Please try again later.")
            return 2, []  # Return 2 when rate limit error occurs
        except self.tweepy.errors.Unauthorized:
            print("Error: Unauthorized access. Check your bearer token.")
            return 1, []  # Return 1 for unauthorized access (no tweets)
        except Exception as e:
            print(f"Unexpected error while checking tweet existence: {e}")
            return 1, []  # Return 1 for any other errors (no tweets)

    def fetch_and_check(self, query, max_results=10):
        """
        Combines fetching and verifying tweets for a given query.

        :param query: Search query string.
        :param max_results: Maximum number of tweets to fetch for verification.
        :return: Weight based on tweet existence or "Not Working" in case of error, and the tweets if any.
        """
        # Call the check_tweet_existence function
        return_code, tweets = self.check_tweet_existence(query, max_results)

        # Allocate dynamic weight or handle scraping issue
        if return_code == 0:
            return 10, tweets  # Return 10 when tweets are found
        elif return_code == 1:
            return 0, []  # Return 0 when no tweets are found
        elif return_code == 2:
            return "Not Working", []  # Return "Not Working" for rate limit or errors


In [15]:
class Agent6Integration:
    def __init__(self, agent1, agent2, agent3, agent4, agent5, agent7):  # Correct constructor name
        self.agent1 = agent1
        self.agent2 = agent2
        self.agent3 = agent3
        self.agent4 = agent4
        self.agent5 = agent5
        self.agent7 = agent7

    def run_agents(self, text, query=None, max_results=10, headline=None, body=None):
        # Run Agent 1 (Special symbols check)
        special_symbols_percent = self.agent1.check_special_symbols(text)

        # Run Agent 2 (Sentiment analysis)
        sentiment = self.agent2.analyze_sentiment(text)

        # Run Agent 3 (Hashtags count)
        hashtags_percent = self.agent3.count_hashtags(text)

        # Run Agent 4 (Clickbait detection)
        clickbait = self.agent4.check_clickbait(text, headline, body)

        # Run Agent 5 (Fake news prediction using BERT)
        real_prob, fake_prob = self.agent5.predict_with_percentage(text)

        # Run Agent 7 (Tweet existence check)
        tweet_result_code, tweets = self.agent7.fetch_and_check(query, max_results)

        # Create the standardized output dictionary
        standardized_outputs = {
            "Agent1_Percentage": special_symbols_percent,
            "Agent2_Binary": 1 if sentiment == 1 else 0,
            "Agent3_Percentage": hashtags_percent,
            "Agent4_Binary": 1 if clickbait else 0,
            "Agent5_Percentage": fake_prob,
            "Agent7_Categorical": tweet_result_code,
        }

        # Create and use Agent6 for applying weights
        agent6 = Agent6()
        weighted_score = agent6.apply_weights(standardized_outputs)

        return weighted_score


# Agent6 - Apply Weights Logic (with explanation)
class Agent6:
    def __init__(self):  # Correct constructor name
        pass

    def apply_weights(self, standardized_outputs):
        # Retrieve the output from Agent 7
        agent7_output = standardized_outputs["Agent7_Categorical"]

        # Define weights based on Agent7 output (if Twitter query results in issues)
        if agent7_output == 2:  # Rate limit exceeded
            weights = {
                "Agent1": 5,
                "Agent2": 15,
                "Agent3": 5,
                "Agent4": 25,
                "Agent5": 45,
                "Agent7": 0,
            }
        else:
            # Assign normal weights if no issues with Agent7
            weights = {
                "Agent1": 5,
                "Agent2": 15,
                "Agent3": 5,
                "Agent4": 20,
                "Agent5": 40,
                "Agent7": 10,
            }

        # Apply weights and calculate the final weighted score
        adjusted_weights = {}
        total_weight_contribution = 0
        print("Individual Weight Contributions:")

        for agent, weight in weights.items():
            if agent == "Agent1":
                contribution = weight * (standardized_outputs["Agent1_Percentage"] / 100)
                adjusted_weights[agent] = contribution
            elif agent == "Agent2":
                contribution = weight * standardized_outputs["Agent2_Binary"]
                adjusted_weights[agent] = contribution
            elif agent == "Agent3":
                contribution = weight * (standardized_outputs["Agent3_Percentage"] / 100)
                adjusted_weights[agent] = contribution
            elif agent == "Agent4":
                contribution = weight * (1 - standardized_outputs["Agent4_Binary"])
                adjusted_weights[agent] = contribution
            elif agent == "Agent5":
                contribution = weight * (standardized_outputs["Agent5_Percentage"] / 100)
                adjusted_weights[agent] = contribution
            elif agent == "Agent7":
                contribution = weight * (1 if agent7_output in [0, 1] else 0)
                adjusted_weights[agent] = contribution

            # Add contribution to total weight
            total_weight_contribution += contribution

            # Print each agent's weight contribution
            print(f"{agent} Weight Contribution: {contribution:.2f}")

        # Display total weight before final weighted score
        print(f"\nTotal Weight Contribution from all Agents: {total_weight_contribution:.2f}")

        # Calculate the final weighted score
        print(f"Final Weighted Score: {total_weight_contribution:.2f}")
        return total_weight_contribution


In [17]:
class Agent6Integration:
    def __init__(self, agent1, agent2, agent3, agent4, agent5, agent7):  # Correct constructor name
        self.agent1 = agent1
        self.agent2 = agent2
        self.agent3 = agent3
        self.agent4 = agent4
        self.agent5 = agent5
        self.agent7 = agent7

    def run_agents(self, text, query=None, max_results=10, headline=None, body=None):
        print("Running Agents...")  # Debugging statement

        # Run Agent 1 (Special symbols check)
        special_symbols_percent = self.agent1.check_special_symbols(text)
        print(f"Agent 1 Output: {special_symbols_percent}")  # Debugging statement

        # Run Agent 2 (Sentiment analysis)
        sentiment = self.agent2.analyze_sentiment(text)
        print(f"Agent 2 Output (Sentiment): {sentiment}")  # Debugging statement

        # Run Agent 3 (Hashtags count)
        hashtags_percent = self.agent3.count_hashtags(text)
        print(f"Agent 3 Output (Hashtags Percentage): {hashtags_percent}")  # Debugging statement

        # Run Agent 4 (Clickbait detection)
        clickbait = self.agent4.check_clickbait(text, headline, body)
        print(f"Agent 4 Output (Clickbait): {clickbait}")  # Debugging statement

        # Run Agent 5 (Fake news prediction using BERT)
        real_prob = self.agent5.predict_true_weight(text)
        fake_prob = 100 - real_prob  # Debugging statement
        print(f"Agent 5 (Fake News Prediction): Real Probability: {real_prob}%, Fake Probability: {fake_prob}%")

        # Run Agent 7 (Tweet existence check)
        tweet_result_code, tweets = self.agent7.fetch_and_check(query, max_results)
        print(f"Agent 7 Output (Tweet Result Code): {tweet_result_code}, Tweets: {tweets}")  # Debugging statement

        # Create the standardized output dictionary
        standardized_outputs = {
            "Agent1_Percentage": special_symbols_percent,
            "Agent2_Binary": 1 if sentiment == 1 else 0,
            "Agent3_Percentage": hashtags_percent,
            "Agent4_Binary": 1 if clickbait else 0,
            "Agent5_Percentage": real_prob,
            "Agent7_Categorical": tweet_result_code,
        }

        # Now use Agent6 to apply weights
        agent6 = Agent6()
        weighted_score = agent6.apply_weights(standardized_outputs)

        return weighted_score


class Agent6:
    def __init__(self):  # Correct constructor name
        pass

    def apply_weights(self, standardized_outputs):
        print("Applying Weights...")  # Debugging statement
        # Retrieve the output from Agent 7
        agent7_output = standardized_outputs["Agent7_Categorical"]

        # Define weights based on Agent7 output (if Twitter query results in issues)
        if agent7_output == 2:  # Rate limit exceeded
            weights = {
                "Agent1": 5,
                "Agent2": 15,
                "Agent3": 5,
                "Agent4": 25,
                "Agent5": 45,
                "Agent7": 0,
            }
        else:
            # Assign normal weights if no issues with Agent7
            weights = {
                "Agent1": 5,
                "Agent2": 15,
                "Agent3": 5,
                "Agent4": 20,
                "Agent5": 40,
                "Agent7": 10,
            }

        # Apply weights and calculate the final weighted score
        adjusted_weights = {}
        total_weight_contribution = 0
        print("Individual Weight Contributions:")

        for agent, weight in weights.items():
            if agent == "Agent1":
                contribution = weight * (standardized_outputs["Agent1_Percentage"] / 100)
                adjusted_weights[agent] = contribution
            elif agent == "Agent2":
                contribution = weight * standardized_outputs["Agent2_Binary"]
                adjusted_weights[agent] = contribution
            elif agent == "Agent3":
                contribution = weight * (standardized_outputs["Agent3_Percentage"] / 100)
                adjusted_weights[agent] = contribution
            elif agent == "Agent4":
                contribution = weight * (1 - standardized_outputs["Agent4_Binary"])
                adjusted_weights[agent] = contribution
            elif agent == "Agent5":
                contribution = weight * (standardized_outputs["Agent5_Percentage"] / 100)
                adjusted_weights[agent] = contribution
            elif agent == "Agent7":
                contribution = weight * (1 if agent7_output in [0, 1] else 0)
                adjusted_weights[agent] = contribution

            # Add contribution to total weight
            total_weight_contribution += contribution

            # Print each agent's weight contribution
            print(f"{agent} Weight Contribution: {contribution:.2f}")

        # Display total weight before final weighted score
        print(f"\nTotal Weight Contribution from all Agents: {total_weight_contribution:.2f}")

        # Calculate the final weighted score
        print(f"Final Weighted Score: {total_weight_contribution:.2f}")
        return total_weight_contribution


# Example usage
if __name__ == "__main__":
    # Initialize agents (you'll need to replace these with real implementations or mocks)
    agent1 = Agent1()  # Assume you have the Agent1 class already
    agent2 = Agent2()  # Assume you have the Agent2 class already
    agent3 = Agent3()  # Assume you have the Agent3 class already
    agent4 = Agent4(20)  # Assume you have the Agent4 class already
    agent5 = Agent5()  # Assume you have the Agent5 class already
    agent7 = Agent7(bearer_token="AAAAAAAAAAAAAAAAAAAAAD8%2BxAEAAAAA4mFW4oKeg5L5Je9VJyQUnLsm28M%3DCYuxaWZVBhKMGthEBSFoCXvk0VLhGPHEqTJIF9q3YCAB1M3eGr")  # Your actual token

    # Integrate agents and Agent6
    agent6_integration = Agent6Integration(agent1, agent2, agent3, agent4, agent5, agent7)

    # Test sentence for classification (user input)
    text = input("Please enter a sentence for classification: ")

    # Optionally, provide additional parameters (query, headline, body)
    query = text
    headline = "The shocking truth about AI"
    body = "This is a must-read article that unveils the shocking truth about AI and technology."

    # Run all agents and get the final weighted score
    weighted_score = agent6_integration.run_agents(text, query, max_results=10, headline=headline, body=body)
    print(f"Final Weighted Score: {weighted_score:.2f}")


Running Agents...
Special Symbols: 7, Total Characters: 436, Percentage: 1.61%
Agent 1 Output: 5
Sentiment Percentage: 29.91%
Agent 2 Output (Sentiment): 4.48575
Hashtags: 0, Total Words: 69, Percentage: 0.00%
Hashtag Percentage below threshold. Weight: 5
Agent 3 Output (Hashtags Percentage): 5
Clickbait Score: 1, Percentage: 25.00%
Reduction Factor: 0.2231, Adjusted Weight: 4.46
Agent 4 Output (Clickbait): 4.462603202968596
Agent 5 (Fake News Prediction): Real Probability: 44.999640583992004%, Fake Probability: 55.000359416007996%
Error: Rate limit exceeded. Please try again later.
Agent 7 Output (Tweet Result Code): Not Working, Tweets: []
Applying Weights...
Individual Weight Contributions:
Agent1 Weight Contribution: 0.25
Agent2 Weight Contribution: 0.00
Agent3 Weight Contribution: 0.25
Agent4 Weight Contribution: 0.00
Agent5 Weight Contribution: 18.00
Agent7 Weight Contribution: 0.00

Total Weight Contribution from all Agents: 18.50
Final Weighted Score: 18.50
Final Weighted Score